# Types and Pattern Matching

## Decomposition

- Let's motivate this with an example; suppose we want to write an interpreter for arithmetic expressions

- For simplicity, let's only deal with 2 types of Expressions; either I get a `Number`, or I get a `Sum`
  - That is, in my evaluator, either I receieve a number, or an expression of a `Sum` of 2 `Numbers`

  ```scala
    trait Expr:
      def isNumber: Boolean
      def isSum: Boolean
      def numValue: Int
      def leftOp: Expr
      def rightOp: Expr

    class Number(n: Int) extends Expr:
      def isNumber = true
      def isSum = false
      def numValue = n
      def leftOp = throw new Error("Number.leftOp")
      def rightOp = throw new Error("Number.rightOp")

    class Sum(e1: Expr, e2: Expr) extends Expr:
      def isNumber = false
      def isSum = true
      def numValue = throw new Error("Sum.numValue")
      def leftOp = e1
      def rightOp = e2
  ```


- Let's try to implement the evaluator `eval`
  ```scala
    def eval(e: Expr): Int = 
      if e.isNumber then e.numValue
      else if e.isSum then eval(e.leftOp) + eval(e.rightOp)
      else throw new Error("Unknown expression " + e)
  ```

- This works, but what if we want to add in operators like multiply? Subtract? Divide?
  - It will become super messy
  - And it will be hell on earth to run the evaluator, because before parsing, you need to check the `Expr` type properly, or you may access members that are not available (e.g. trying to access `numValue` in a Sum)

  ```scala
  class Prod(e1: Expr, e2: Expr) extends Expr

  class Var(x: String) extends Expr

  ```

- Exercise: To integrate Prod and Var into the hierarchy, how many new method
definitions do you need?
  - isProd
  - isVar
  - [Optional] varValue 
  - So depending on implementation, you are adding more than 10 new methods for a basic change!

### (Bad) Solution 1: Type Tests and Type Casts

- In `eval` You can deliberately check the type of the `Expr` before doing the evaluation

- This is both ugly and not scalable

  ```scala
    def eval(e: Expr): Int = {
      if e.isInstanceOf[Number] then
        e.asInstanceOf[Number].numValue
      else if e.isInstanceOf[Sum] then
        eval(e.leftOp) + eval(e.rightOp)
      else
        throw Error("Unknown Expression " + e)
    }
  ```

### Solution 2: Object-Oriented Decomposition

- Instead of a single eval function, you could add `eval` as an abstract method to the `Expr` trait, then initialise it in each of the subclasses

  ```scala
    trait Expr:
      def eval: Int

    class Number(n: Int) extends Expr:
      def eval: Int = n

    class Sum(e1: Expr, e2: Expr) extends Expr:
      def eval: Int = e1.eval + e2.eval
  ```

- This is known as object-oriented decomposition, where you mix the data object with the relevant operations

- Pros: To add a new class of data, you can just add a single class

  ```scala
    class Product(e1: Expr, e2: Expr) extends Expr:
      def eval: Int = e1.eval * e2.eval
  ```

- Cons: Suppose you wish to add an operation that doesn't just work on a single object
  - e.g. I want to simplify `a*b + a*c` into `a * (b+c)`
  - This operation is non-local (i.e. involves more than 1 object)
  - Therefore, encapsulating the `eval` method within individual classes will still require some sort of eval type check, which we desperately want to avoid

## Pattern Matching

- Recall from the previous section, we are trying to find a way to access objects in a class hierachy

- In the last section, we tried
  - Adding common methods to all classes ==> led to quadratic explosion
  - Adding type tests ==> Non maintainable code + potentially unsafe
  - Object-oriented decomposition ==> Couples data and operations, all classes affected when adding new method

- Let's try to generalise what we want to do; basically we need the same method to do different things according to the class it is presented
  - In this case, the type checks/decomposition etc are all trying to **reverse** the construction process (i.e. figure out which subclass was used and what the arguments were)

- Thankfully in Scala, there is an idiomatic way to do this via **case classes**

- This can be applied in 2 steps. First, define the relevant `case class`. Then, use the keyword `match` to check if the input matches the `case class`

  ```scala
    trait Expr
    case class Number(n: Int) extends Expr
    case class Sum(e1: Expr, e2: Expr) extends Expr

    def eval(e: Expr): Int = e match
      case Number(n) => n
      case Sum(e1, e2) => eval(e1) + eval(e2)
      case _ => throw new Error("wtf is this")
  ```

- Patterns must match one of:
  - constructors e.g. `Number`, `Sum`
  - variable e.g. `e1`
  - wildcard pattern e.g. `_`
  - constants e.g. `true`
  - type tests e.g. `n: Number`

- The evaluation resolves in the following order
  ```scala
    //1
    eval(Sum(Number(1), Number(2))) 

    //2
    Sum(Number(1), Number(2)) match
      case Number(n) => n
      case Sum(e1, e2) => eval(e1) + eval(e2)

    //3
    eval(Number(1)) + eval(Number(2))

    //4
    Number(1) match
      case Number(n) => n
      case Sum(e1, e2) => eval(e1) + eval(e2)
    + eval(Number(2))

    //4
    Number(1) match
      case Number(n) => n
      case Sum(e1, e2) => eval(e1) + eval(e2)
    + eval(Number(2))

    //5
    1 + eval(Number(2))

    //6
    3
  ```



- As a convenient alternative, you can place the `eval` function under the `Expr` trait
  ```scala
  trait Expr:
    def eval: Int = this match
      case Number(n) => n
      case Sum(e1, e2) => e1.eval + e2.eval
  ```

### Exercise

- Write a function show that uses pattern matching to return the
representation of a given expressions as a string.

  ```scala
    def show(e: Expr): String = ???
  ```

- See `2-slides.scala`

### Exercise 2

- Add case classes Var for variables x and Prod for products x * y as discussed previously.

- Change your show function so that it also deals with products. Pay attention you get operator precedence right but to use as few parentheses as possible

- Examples
  - Sum(Prod(2, Var("x")), Var("y")) ==> "2 * x + y"
  - Prod(Sum(2, Var("x")), Var("y")) ==> "(2 + x) * y"

- See 2-slides.scala